In [8]:
import pandas as pd
import datetime
import numpy as np
import matplotlib.pyplot as plt
import time
import datetime
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier

In [9]:
data = pd.read_csv('EOCC_New_data_with_zero_updated_columns.csv')

In [10]:
data.head()

,key,CAR NUMBER,LOCATION,CONDITION CODE,is_new,RESPONSE CODE,WHY MADE,PART APPLIED DATE,UPDATED PART REPAIR DATE,weeks_in_service,...,BOCT,CN,CPRS,CSXT,FXE,KCS,KCSM,NSS,ST,UP
0,83500A,83500,A,1,1,0,0,3/23/2018,11/15/2018,33.86,...,0,3,0,3,0,0,0,4,0,15
1,83500B,83500,B,1,1,0,0,6/6/2018,11/15/2018,23.14,...,0,3,0,3,0,0,0,4,0,15
2,83503A,83503,A,3,0,0,0,12/19/2014,11/15/2018,203.86,...,0,6,0,6,0,0,0,5,0,23
3,83503B,83503,B,3,0,0,0,10/20/2016,11/15/2018,108.00,...,0,6,0,6,0,0,0,5,0,23
4,83505A,83505,A,1,1,0,0,6/22/2017,11/15/2018,73.00,...,0,9,3,12,0,1,0,2,0,28


In [11]:
data['CAR_CLASS'] = pd.factorize(data.CAR_CLASS)[0]
data['EQUIP_FEAT_DESC'] = pd.factorize(data.EQUIP_FEAT_DESC)[0]
data['car_length'] = pd.factorize(data.car_length)[0]
data['car_tonnage'] = pd.factorize(data.car_tonnage)[0]
data['BUILDER_NAME'] = pd.factorize(data.BUILDER_NAME)[0]

In [12]:
data1 = pd.DataFrame(data['CAR NUMBER'].unique())
data1.columns=['CAR NUMBER']
data2=pd.DataFrame(np.random.rand(len(data['CAR NUMBER'].unique())))
data2.columns=['RandomNum']
data1['RandomNum']=pd.DataFrame(np.random.rand(len(data['CAR NUMBER'].unique())))
data_w_rand=pd.merge(data,data1,on='CAR NUMBER',how='left')

data_new =data_w_rand.drop(['CAR NUMBER','key','LOCATION','RESPONSE CODE','WHY MADE','PART APPLIED DATE','UPDATED PART REPAIR DATE','UPDATED PART REPAIR DATE','BUILT DATE'],axis =1)

#train and test  
data_train = data_new[data_new['RandomNum'] <= 0.75]
data_test = data_new[data_new['RandomNum'] > 0.75]

In [13]:
df_no_fail_test = data_test[data_test['TARGET']== 0]
df_fail_train = data_train[data_train['TARGET']== 1]
df_fail_test =  data_test[data_test['TARGET']== 1]

In [14]:
y_test = df_fail_test['TARGET']
x_test = df_fail_test.drop(['TARGET'],axis =1)
y_train = df_fail_train['TARGET']
x_train = df_fail_train.drop(['TARGET'],axis =1)

In [15]:
y_test_no_fail = df_no_fail_test['TARGET']
x_test_no_fail = df_no_fail_test.drop(['TARGET'],axis =1)

In [16]:
from sklearn.ensemble import IsolationForest
clf = IsolationForest(n_estimators = 500, bootstrap=True,n_jobs=3).fit(x_train)

C:\Users\arian\Anaconda3\lib\site-packages\sklearn\ensemble\iforest.py:237: FutureWarning: default contamination parameter 0.1 will change in version 0.22 to "auto". This will change the predict method behavior.
  FutureWarning)
C:\Users\arian\Anaconda3\lib\site-packages\sklearn\ensemble\iforest.py:247: FutureWarning: behaviour="old" is deprecated and will be removed in version 0.22. Please use behaviour="new", which makes the decision_function change to match other anomaly detection algorithm API.
  FutureWarning)


In [17]:
y_pre_train = clf.predict(x_train)
y_pre_test = clf.predict(x_test)
y_pre_nofail_test = clf.predict(x_test_no_fail)

C:\Users\arian\Anaconda3\lib\site-packages\sklearn\ensemble\iforest.py:415: DeprecationWarning: threshold_ attribute is deprecated in 0.20 and will be removed in 0.22.
  " be removed in 0.22.", DeprecationWarning)
C:\Users\arian\Anaconda3\lib\site-packages\sklearn\ensemble\iforest.py:415: DeprecationWarning: threshold_ attribute is deprecated in 0.20 and will be removed in 0.22.
  " be removed in 0.22.", DeprecationWarning)
C:\Users\arian\Anaconda3\lib\site-packages\sklearn\ensemble\iforest.py:415: DeprecationWarning: threshold_ attribute is deprecated in 0.20 and will be removed in 0.22.
  " be removed in 0.22.", DeprecationWarning)


In [ ]:
## Accuracy for test: failure data

In [18]:
1 + np.sum(y_pre_test[y_pre_test == -1])/len(y_pre_test)

0.8817204301075269

In [19]:
## Accuracy for train: : failure data

In [20]:
1 + np.sum(y_pre_train[y_pre_train == -1])/len(y_pre_train)

0.8994974874371859

In [22]:
1 - np.sum(y_pre_nofail_test[y_pre_nofail_test == 1])/len(y_pre_nofail_test)

0.11700526711813397